In [1]:

from acdesign.atmosphere import Atmosphere
from acdesign.solar_wing import SolarWing
from acdesign.airfoils.polar import UIUCPolar, available_sections

import numpy as np
import pandas as pd
section = UIUCPolar.local("CLARKYB")

wings = dict (
    Straight = SolarWing.straight(28, 2, section),
    Double_Taper = SolarWing.double_taper(21, 15, 2, section),
    Straight_to_Elliptical = SolarWing.straight_to_elliptical(19, 18, 2, section),
)

pd.DataFrame({k: wing.data() for k, wing in wings.items()}).round(2)

,Straight,Double_Taper,Straight_to_Elliptical
npanels,56.00,56.00,56.00
b,3.74,4.78,4.95
S,1.35,1.28,1.29
SMC,0.36,0.27,0.26
AR,10.39,17.90,19.10
TR,1.00,0.62,0.00


In [2]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,)
wings["Straight"].plot(fig, row=1, col=1)
wings["Double_Taper"].plot(fig, row=2, col=1)
wings["Straight_to_Elliptical"].plot(fig, row=3, col=1)


fig.update_layout(
    template="plotly_white", 
    yaxis=dict(scaleanchor="x"),
    yaxis2=dict(scaleanchor="x"),
    yaxis3=dict(scaleanchor="x"),
    legend=dict(visible=False),
    width=900, height=500,
    margin=dict(l=0, r=0, t=0, b=0)
)
fig.show()

In [3]:
airspeeds = np.linspace(8, 20, 13)

maximum_masses = pd.DataFrame({
    k: wing.max_mass(airspeeds, atm=Atmosphere.alt(0), min_mass=3, max_mass=15) for k, wing in wings.items()
})

maximum_masses_alt = pd.DataFrame({
    k: wing.max_mass(airspeeds, atm=Atmosphere.alt(9000), min_mass=3, max_mass=15) for k, wing in wings.items()
})


In [4]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
fig = go.Figure()
for j, df in enumerate([maximum_masses, maximum_masses_alt]):
    for i, col in enumerate(df.columns):
        fig.add_trace(go.Scatter(
            x=airspeeds,
            y=df[col],
            name=col,
            line=dict(dash=["solid", "dash"][j], color=px.colors.qualitative.Plotly[i]  ),

        ))
fig

In [8]:
from acdesign.airfoils.polar import available_sections
from pathlib import Path
if Path("examples/solar_plane/section_selection.csv").exists():
    section_selection = pd.read_csv("examples/solar_plane/section_selection.csv", index_col=[0,1])
else:


    results = {}
    mass = np.linspace(5, 15, 11)
    v = np.linspace(10, 25, 16)

    mass, v = np.meshgrid(mass, v)
    mass, v = mass.flatten(), v.flatten()


    for section in available_sections():
        wing =  SolarWing.straight_to_elliptical(19, 18, 2, UIUCPolar.local(section))
        results[section] = wing.run(mass, v, Atmosphere.alt(0))

    section_selection = pd.concat(results)
    section_selection.to_csv("examples/solar_plane/section_selection.csv")

In [9]:
section_selection

ss = section_selection.loc[section_selection.power < section_selection.solar_power].reset_index().rename(columns={"level_0": "section"})

fig = go.Figure()

airspeed=20

ss.loc[ss.airspeed==airspeed].groupby("section").max().mass
px.bar(
    ss.loc[ss.airspeed==airspeed].groupby("section").max().mass,
).show()

for (section), df in ss.groupby("section"):
    masses = df.groupby("airspeed").max().mass
    fig.add_trace(go.Scatter(
        x=masses.index,
        y=masses.values,
        name=section,
        mode="lines"
    ))

fig
#    df = _df.groupby("mass").max.airspeed

#    print(df)

